In [1]:
!pip install pillow torchvision boto3 pandas

In [2]:
import pandas as pd
import boto3

In [3]:
import boto3
from PIL import Image
from torchvision import transforms
import os
import io

In [4]:
# AWS S3 클라이언트 설정
s3 = boto3.client('s3')
bucket_name = 'smwu-cv-data'
ok_folder_path = 'data/두부/OK/'

In [5]:
# 전처리를 위한 변환 정의
preprocessing_transforms = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ColorJitter(brightness=0.5, contrast=0.5),  # Adjust brightness/contrast
    transforms.RandomRotation(15),  # Apply random rotation
    transforms.RandomHorizontalFlip(),  # Apply horizontal flip
    transforms.GaussianBlur(3)  # Add Gaussian blur
])

In [6]:
# S3에서 객체 목록 가져오기
def list_all_objects(bucket_name, prefix):
    """S3에서 주어진 폴더 내 모든 객체 목록 가져오기"""
    all_objects = []
    result = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' not in result:
        return all_objects

    all_objects.extend(result['Contents'])

    while result.get('IsTruncated'):
        continuation_token = result['NextContinuationToken']
        result = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
        all_objects.extend(result['Contents'])

    return all_objects

In [7]:
# S3에서 이미지 다운로드
def download_image_from_s3(bucket_name, file_key):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        img_data = response['Body'].read()
        return Image.open(io.BytesIO(img_data)).convert('RGB')
    except Exception as e:
        print(f"Error downloading {file_key}: {e}")
        return None

In [8]:
# 전처리 수행
def preprocess_image(img):
    """이미지 전처리 수행"""
    return preprocessing_transforms(img)

In [9]:
# S3로 업로드
def upload_to_s3(local_folder, bucket_name, s3_folder):
    """로컬 디렉토리에서 S3로 업로드"""
    for file_name in os.listdir(local_folder):
        file_path = os.path.join(local_folder, file_name)
        s3_key = f"{s3_folder}/{file_name}"
        s3.upload_file(file_path, bucket_name, s3_key)
        print(f"Uploaded {file_name} to s3://{bucket_name}/{s3_key}")

In [10]:
# 데이터 전처리 및 저장
def preprocess_and_save(bucket_name, folder_path, save_path):
    """S3에서 다운로드 -> 전처리 -> 로컬 저장"""
    os.makedirs(save_path, exist_ok=True)
    objects = list_all_objects(bucket_name, folder_path)

    for obj in objects:
        file_key = obj['Key']
        if file_key.endswith('.bmp'):
            img = download_image_from_s3(bucket_name, file_key)
            if img is None:
                continue

            # 전처리 수행
            processed_img = preprocess_image(img)
            output_path = os.path.join(save_path, os.path.basename(file_key))
            processed_img.save(output_path)
            print(f"Processed and saved: {output_path}")

    print("데이터 전처리 및 로컬 저장 완료.")

In [11]:
# 실행 경로 설정
local_save_path = '/home/ec2-user/SageMaker/processed_images/OK'

In [12]:
# 실행
preprocess_and_save(bucket_name, ok_folder_path, local_save_path)

print("데이터 전처리 작업 완료. 전처리된 데이터는 SageMaker Notebook의 지정된 경로에 저장되었습니다.")

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/0.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/10.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/100.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1000.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1001.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1002.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1003.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1004.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1005.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1006.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1007.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1008.bmp
Processed and saved: /home/ec2-use

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/11.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/110.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1100.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1101.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1102.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1103.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1104.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1105.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1106.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1107.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1108.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1109.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/111.bmp
Processed and saved: /home/ec

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1201.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1202.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1203.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1204.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1205.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1206.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1207.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1208.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1209.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/121.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1210.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1211.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1212.bmp
Processed and saved: /home

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1303.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1304.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1305.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1306.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1307.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1308.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1309.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/131.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1310.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1311.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1312.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1313.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1314.bmp
Processed and saved: /home

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1402.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1404.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1405.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1406.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1407.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1408.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1409.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/141.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1410.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1411.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1412.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1413.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/1414.bmp
Processed and saved: /home

...

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/5094.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/5095.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/5096.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/5097.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/5098.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/5099.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/51.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/510.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/511.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/512.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/513.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/514.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/515.bmp
Processed and saved: /home/ec2-us

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/604.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/605.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/606.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/607.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/608.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/609.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/61.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/610.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/611.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/612.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/613.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/614.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/615.bmp
Processed and saved: /home/ec2-user/Sag

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/705.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/706.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/707.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/708.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/709.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/71.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/710.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/711.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/712.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/713.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/714.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/715.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/716.bmp
Processed and saved: /home/ec2-user/Sag

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/805.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/806.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/807.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/808.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/809.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/81.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/810.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/811.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/812.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/813.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/814.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/815.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/816.bmp
Processed and saved: /home/ec2-user/Sag

Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/906.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/907.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/908.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/909.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/91.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/910.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/911.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/912.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/913.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/914.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/915.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/916.bmp
Processed and saved: /home/ec2-user/SageMaker/processed_images/OK/917.bmp
Processed and saved: /home/ec2-user/Sag